# A two stage stochastic program

Assume that you are warming your house with fuel. Currently fuel is at 750€/1000 liters. 
You can store up to 1000l for the winter, but you don't know yet how much you will use, and how much it will cost to buy fuel during winter.

We assume that there are $4$ scenarios, with probabilities $p$, futur fuel cost $c$ and demand $d$ given afterwards.
We want to minimize the expected costs of warming yourself during winter.

In [11]:
using JuMP, Clp

S = 4 
p = [0.1 0.4 0.4 0.1]
c0 = 750
c1 = [700 770 900 900]
d = [500 700 800 1000]

1x4 Array{Int64,2}:
 500  700  800  1000

## Question 1

Solve the corresponding 2 stage program. Give the optimal value, and optimal controls.

In [88]:
m_2S = Model(solver = ClpSolver())
@variable(m_2S, u1)
@variable(m_2S, u2[1:4])
@variable(m_2S, cout[1:4])
@constraint(m_2S, u1 <=1000)
for i = 1:4
    @constraint(m_2S, u1+u2[i] >= d[i])
end
for i = 1:4
    @constraint(m_2S, u2[i] >= 0)
end
@constraint(m_2S, u1 >= 0)
for i=1:4
    @constraint(m_2S, cout[i] >= u1*c0 + c1[i]*u2[i])
end
@objective(m_2S, Min, sum(p*cout))
solve(m_2S)
value_2S = getvalue(sum(p*cout))

588000.0

Resolution gives us optimal value for u1 = 700 L

In [74]:
getvalue(u1)

700.0

## Question 2

Solve the open-loop version of this problem. Give the optimal value, and optimal controls.
Check the inequality between the value of the open-loop problem and 2-stage problem.

In [89]:
m_OL = Model(solver = ClpSolver())
@variable(m_OL, u0)
@variable(m_OL, u1)
@variable(m_OL, cout[1:4])
@constraint(m_OL, u0 <=1000)
for i =1:4
    @constraint(m_OL, u0+u1 >= d[i])
end

@constraint(m_OL, u0 >= 0)
@constraint(m_OL, u1 >= 0)

for i=1:4
    @constraint(m_OL, cout[i] >= u0*c0 + c1[i]*u1)
end
@objective(m_OL, Min, sum(p*cout))
solve(m_OL)
value_OL = getvalue(sum(p*cout))

750000.0

In this case, as we don't allow u1 to be chosen according to the need in fuel, we decide at stage 0 how much fuel we need to get. As the fuel price is exepected to rise above c0, the solution is to buy as much fuel as we can at stage 0, i.e. u0 = 1000

In [80]:
getvalue(u0)

1000.0

## Question 3

Solve the anticipative version of this problem. Give the optimal value, and optimal controls. Check the inequality between the value of the anticipative problem and 2-stage problem.

In [90]:
m_a = Model(solver = ClpSolver())
@variable(m_a, u0[1:4])
@variable(m_a, u1[1:4])
@variable(m_a, cout[1:4])
for i =1:4
    @constraint(m_a, u0[i] <=1000)
end
for i = 1:4
    @constraint(m_a, u0[i]+u1[i] >= d[i])
end
for i = 1:4
    @constraint(m_a, u0[i] >= 0)
    @constraint(m_a, u1[i] >= 0)
    
end

for i=1:4
    @constraint(m_a, cout[i] >= u0[i]*c0 + c1[i]*u1[i])
end
@objective(m_a, Min, sum(p*cout))
solve(m_a)
value_a = getvalue(sum(p*cout))

560000.0

In [110]:
getvalue(p*cout)

1-element Array{Float64,1}:
 560000.0

As expected we each time buy exactly the quantity of fuel needed in each sceanario, at the best price. We have the inequality :

$$value_a \leq value_{2S} \leq value_{OL} $$

## Question 4

We call (P_mean) the problem where cost and demand are replaced by their expectation. Solve this problem, giving value and first and second stage optimal control.

Evaluate the value of this first stage control, that is the expected cost of using this first stage control with adapted recourse.

Compare both values to the precedents problems.

In [101]:
m_mean = Model(solver = ClpSolver())
d_mean = sum(p*d)
c1_mean = mean(c1)
@variable(m_mean, u0)
@variable(m_mean, u1)
@constraint(m_mean,u0 <=1000)
@constraint(m_mean, u0 >= 0 )
@constraint(m_mean, u1 >= 0)
@constraint(m_mean, u0 + u1 >= d_mean)
@objective(m_mean, Min, c0*u0 + c1_mean * u1)
solve(m_mean)

LoadError: DimensionMismatch("matrix A has dimensions (1,4), matrix B has dimensions (1,4)")

In [122]:
getvalue(u0)
#Expected value u0 = 750

750.0

Now evaluating the expected cost of using such a method to estimate the quantity of fuel needed :

In [121]:
u1 = max(0,d - getvalue(u0))
average_cost =0
for i= 1:4
    average_cost += p[i] * (u1[i]*c1[i] + getvalue(u0)*c0)
end
print(average_cost)

603000.0

We then have this inequality, reflecting the smart use of information it is possible to make

$$value_a \leq value_{2S} \leq value_{average} \leq value_{OL} $$